In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np
import plotly.graph_objects as go
from data.simulation.simulator import Simulator
from sampler import CVXSampler
from learner import SVMLearner
template = "plotly_white"

# Initialize variables

In [7]:
# Initialize
simulator = Simulator("moon", noise=0.2)
learner = SVMLearner()
npr = np.random.RandomState(123)
sigma = 5 
assert sigma > 1.

# Generating Data

In [8]:
# Dataset
N = 100             # Number of simulation data
input_dim = 2       # Feature dimension
train_X, train_y = simulator.simulate(N, input_dim)
labeled_mask = np.zeros(N).astype(np.bool)
labeled_ratio = 0.1
labeled_mask[:int(labeled_ratio * N)] = True

test_X, test_y = simulator.simulate(N, input_dim)

In [9]:
def vis_scatter(X, color):
    # Visualize the data
    fig = go.Figure(data=go.Scatter(x=X[:, 0], 
                                    y=X[:, 1], 
                                    mode='markers', 
                                    marker=dict(color=color), 
                                    marker_line_width=2))
    fig.update_layout(template=template)
    fig.update_xaxes(showline=True, linewidth=1.5, linecolor='Black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1.5, linecolor='Black', mirror=True)
    fig.update_layout(width=800, height=600)
    fig.show()

# Visaulize train data
vis_scatter(train_X, train_y)

In [10]:
def eval_acc(X, y, vis=False):
    prob = learner.predict_proba(X)
    pred = prob.argmax(1)
    
    acc = (pred == y).mean()
    print("accuracy: {:.2f}".format(acc))
    
    if vis:
        vis_scatter(X, pred)
    return acc

In [11]:
learner.fit(train_X[labeled_mask], train_y[labeled_mask])
acc = eval_acc(test_X, test_y, vis=True)

accuracy: 0.80


## Select the data to label next

In [12]:
cvx_sampler = CVXSampler(train_X, labeled_mask, K=2, sigma=sigma, alpha=1)
idx_to_label = cvx_sampler.sample(None, learner)

Solving Problem
Takes 40.63 sec


In [13]:
cur_labeled_mask = np.copy(labeled_mask)
cur_labeled_mask[idx_to_label] = True
learner.fit(train_X[cur_labeled_mask], train_y[cur_labeled_mask])
acc = eval_acc(test_X, test_y, vis=True)

IndexError: arrays used as indices must be of integer (or boolean) type

In [27]:
from experiment_manager import ExperimentParams

params = ExperimentParams()
params.N = 100

params.save("test.json")

new_params = ExperimentParams()
new_params.load("test.json")

new_params

In [28]:
new_params.N

100

In [29]:
new_params.sha

'986a7abdbaa79c862927006dc5a0364b4bf8c676'